In [1]:
# ! conda install -c conda-forge python-annoy
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import os, sys, pickle, glob, gc
from collections import Counter
import itertools

import gensim
from annoy import AnnoyIndex

In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'

In [3]:
TOPN_candidate = 50
note = 'covisit_20_20_20_covwgt'
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [4]:
def load_df(path):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(path)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [5]:
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../output/newSplited/savedModel/otto_aid2vec_5d.bin', binary=True)

In [6]:
aid2idx = {int(aid): i for i, aid in enumerate(aid2vec_model.index_to_key)}
index = AnnoyIndex(5, 'euclidean')
for _, idx in aid2idx.items():
    index.add_item(idx, aid2vec_model.vectors[idx])
index.build(10)

True

In [7]:
def calW2vSimilarity(x):
    return index.get_distance(aid2idx[x.aid_x], aid2idx[x.aid_y])

# Load candidate

In [8]:
SETS = [2, 3]
for SET in SETS:
    candidatesSavePath = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{note}/'
    candidatesSavePath_w2vSim = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{note}_w2vSim/'
    ! mkdir {candidatesSavePath_w2vSim}
    predTypes = ['clicks', 'carts', 'orders']
    for predType in predTypes:
        candidate = pd.read_parquet(candidatesSavePath + f'{predType}Candidate.pqt')
        if SET == 2:
            val_A = pd.read_parquet('../input/splited/test.parquet')
        elif SET == 3:
            val_A = load_df('../input/parquets/test_parquets/*')
            # val_A = val_A.drop_duplicates((['session', 'aid']))
        df = val_A.merge(candidate, on='session')
        w2vSim = df[['aid_x', 'aid_y']].drop_duplicates(['aid_x', 'aid_y'])
        w2vSim['w2vSim'] = w2vSim.progress_apply(lambda x: calW2vSimilarity(x), axis=1)
        df = df.merge(w2vSim, on=['aid_x', 'aid_y'], how='left')
        df = df.groupby(['session', 'aid_y']).w2vSim.mean().reset_index()
        df.columns = ['session', 'aid', 'w2vSim']
        candidate = candidate.merge(df, on=['session', 'aid'], how='left')
        candidate.to_parquet(candidatesSavePath_w2vSim + f'{predType}Candidate.pqt')

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

mkdir: cannot create directory ‘../output/newSplited/candidates_set3_top50_covisit_20_20_20_covwgt_w2vSim/’: File exists


A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [9]:
candidatesSavePath_w2vSim

'../output/newSplited/candidates_set3_top50_covisit_20_20_20_covwgt_w2vSim/'

In [ ]:
similarity